# Generate Individual Metadata Files

First we have the necessart imports. These are largely encapsulated inside of the automated metadata generation (`amg`) library. THe only external things we import are `json` (so that we can write out STAC files).

In [1]:
import json

from amg.isismetadata import IsisMetadata
from amg.fgdcmetadata import FGDCMetadata, EquirectangularFgdcParser, PolarStereoGraphicFgdcParser
from amg.gdalmetadata import GDALMetadata
from amg.formatters.stac_formatter import to_stac
from amg.formatters.fgdc_formatter import to_fgdc
from amg import UnifiedMetadata


## Step I: Build the Inputs

In [5]:
#Define Inputs
fgdc = FGDCMetadata('../templates/europa_individual_l2_fgdc.xml', proj='equirect')
gd = GDALMetadata('/archive/projects/europa/GLL_FinProducts/10ESGLOBAL01/Lev2/s0413742778.equi.cub')
imd = IsisMetadata('/archive/projects/europa/GLL_FinProducts/10ESGLOBAL01/Lev2/s0413742778.equi.cub')


# Define overrides
overrides = {'license': 'PDDL-1.0',
             'missions':['Voyager 1', 'Voyager 2', 'Galileo'],
             'doi':'https://doi.org/10.5066/P9VKKK7C',
             'href':'https://asc-jupiter.s3-us-west-2.amazonaws.com/europa/individual_l2'}

# Define mappings
mappings = {'bbox':IsisMetadata, }

The code above might look really busy at first inspection. Here, each step is described in greater detail to help provide context about how `amg` works.

First, a FGDC metadata template is input. The projection (`equirect` in this case) is manually specified so that the template can completely omit a projection section. Next, the spatial data are input both as ISIS cubes and as GDAL files. This is because each library offers different metadata items. These three inputs (`fgdc`, `gd`, and `imd`) contain all of the information needed to output compliant metadata in FGDC and STAC formats.
```
fgdc = FGDCMetadata('europa_individual_l2_fgdc.xml', proj='equirect')
gd = GDALMetadata('/archive/projects/europa/GLL_FinProducts/10ESGLOBAL01/Lev2/s0413742778.equi.cub')
imd = IsisMetadata('/archive/projects/europa/GLL_FinProducts/10ESGLOBAL01/Lev2/s0413742778.equi.cub')
```

Next overrides are defined. An override is a way for the user to explicitly set a field in the output metadata. Perhaps the field is not availabe in the inputs, or perhaps the user wishes to force a specific value that is different from what is parsed from the input files above. In the example here, none of the overrides (`license`, `missions`, `doi`, or `href`) are available from one of the input sources above.

```
overrides = {'license': 'PDDL-1.0',
             'missions':['Voyager 1', 'Voyager 2', 'Galileo'],
             'doi':'https://doi.org/10.5066/P9VKKK7C',
             'href':'https://asc-jupiter.s3-us-west-2.amazonaws.com/europa/individual_l2'}
```

Finally, mappings are defined. A mapping tells the parsers to get a metadata value from a specific input. In this case, the `bbox` or bounding box is going to be explicitly parsed from the IsisMetadata (`imd` above). Left to it's own devices, the parser could get the `bbox` from either the Isis input or the GDAL input.
```
mappings = {'bbox':IsisMetadata, }
```

-----------------------------------------
## Step II: Smash it together!

In [6]:
# Create a unified metadata object
record = UnifiedMetadata([fgdc, gd, imd], overrides=overrides, mappings=mappings)


Above we take the inputs, overrides, and mappings (all described above) and push them into a single, unified metadata object. This `record` is able to perform lookups on all of the input objects to find attributes that are needed to generate metadata.

For anyone interested the unified metadata object does the folling on a `__getattr__` call:

1. Check the overrides to see if the key is in that dict. If so, return the override.
1. Check the mappings. If the key is in the mapping, run get the attribute off of the specified object.
1. Iterate over the input objects and attempt to get the attribute off of any of the inputs.

---------------------------
# Step III: Reformat

In [7]:
# Generate FGDC metadata
fgdc_md = to_fgdc(record)
with open('equirectangular_output.xml', 'w') as f:
    f.write(fgdc_md)
    
# Generate STAC metadata
stac_md = to_stac(record)
with open('equirectangular_output.json', 'w') as f:
    json.dump(stac_md.to_dict(), f, indent=2)

/work/users/jlaura/github/automated-metadata-generation/amg/__init__.py:31: UserWarning: Unable to find the attribute "instruments" in any data source.
  warnings.warn(f'Unable to find the attribute "{attr}" in any data source.')
/work/users/jlaura/github/automated-metadata-generation/amg/__init__.py:31: UserWarning: Unable to find the attribute "gsd" in any data source.
  warnings.warn(f'Unable to find the attribute "{attr}" in any data source.')


Above, we output metadata in the format desired. The library currently has 2 formatters (`to_fgdc` and `to_stac`) that take the unified metadata object, perform some parsing, and then output standards compliant metadata. The libraries that we depend on to parse the metadata include validators, so the outputs are being validated (more or less accurately) before they are written to disk.

Here you can also see that two of the desired attributes (in this case `instruments` and `gsd`) were not available in the input files. The user is only warned the first time this happens, so a second run would silently succeed.

_________________________
## All in one

Below is an all in one example on some polar stereographic data. The structure is identical to the above. The difference is that the input dataset is different.

In [ ]:
# Polar
fgdc = FGDCMetadata('../templates/europa_individual_l2_fgdc.xml', proj='polarst')
gd = GDALMetadata('/archive/projects/europa/GLL_FinProducts/10ESGLOBAL01/Lev2/s0413742778.spola.cub')
imd = IsisMetadata('/archive/projects/europa/GLL_FinProducts/10ESGLOBAL01/Lev2/s0413742778.spola.cub')


overrides = {'license': 'PDDL-1.0',
             'missions':['Voyager 1', 'Voyager 2', 'Galileo'],
             'doi':'https://doi.org/10.5066/P9VKKK7C',
             'href':'https://asc-jupiter.s3-us-west-2.amazonaws.com/europa/individual_l2'}

record = UnifiedMetadata([fgdc, gd, imd], overrides=overrides, mappings={'bbox':IsisMetadata, })

fgdc_md = to_fgdc(record)
with open('polar_output.xml', 'w') as f:
    f.write(fgdc_md)
    
# Generate STAC metadata
stac_md = to_stac(record)
with open('polar_output.json', 'w') as f:
    json.dump(stac_md.to_dict(), f, indent=2)